In [2]:
import folium
import pandas as pd
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

## Task 1: Mark all launch sites on a map

In [3]:
# Read in dataset
df = pd.read_csv('dataset_p2.csv')

df.head()


,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude,Outcomes_binary,Class
0,6,2010-06-04,Falcon 9,6123.547647,LEO,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0003,-80.577366,28.561857,0,0
1,8,2012-05-22,Falcon 9,525.000000,LEO,CCSFS SLC 40,None None,2,False,False,False,NaN,1.0,0,B0005,-80.577366,28.561857,0,0
2,10,2013-03-01,Falcon 9,677.000000,ISS,CCSFS SLC 40,None None,3,False,False,False,NaN,1.0,0,B0007,-80.577366,28.561857,0,0
3,11,2013-09-29,Falcon 9,500.000000,PO,VAFB SLC 4E,False Ocean,4,False,False,False,NaN,1.0,0,B1003,-120.610829,34.632093,0,0
4,12,2013-12-03,Falcon 9,3170.000000,GTO,CCSFS SLC 40,None None,5,False,False,False,NaN,1.0,0,B1004,-80.577366,28.561857,0,0


In [13]:
# Selecting relevent columns
df_spacex = df[['LaunchSite', 'Longitude', 'Latitude', 'Class']]

df_launch_sites = df_spacex.groupby(['LaunchSite'], as_index=False).first()
df_launch_sites = df_launch_sites[['LaunchSite', 'Longitude', 'Latitude']]
df_launch_sites

,LaunchSite,Longitude,Latitude
0,CCSFS SLC 40,-80.577366,28.561857
1,KSC LC 39A,-80.603956,28.608058
2,VAFB SLC 4E,-120.610829,34.632093


In [14]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)
site_map

In [15]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color="#0400d3", fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map

In [16]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
for i, row in df_launch_sites.iterrows():
    coords = [row['Latitude'], row['Longitude']]
    circle = folium.Circle(coords, radius=1000, color="#00eeff", fill=True).add_child(folium.Popup(row['LaunchSite']))


    marker = folium.Marker(
        coords, 
        icon = DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html= '<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % f"{row['LaunchSite']}",
        )
    )   


    site_map.add_child(circle)
    site_map.add_child(marker)

site_map

# Task 2: Mark the success/failed launches for each site on the map


In [17]:
df_spacex.tail()

,LaunchSite,Longitude,Latitude,Class
85,KSC LC 39A,-80.603956,28.608058,1
86,KSC LC 39A,-80.603956,28.608058,1
87,KSC LC 39A,-80.603956,28.608058,1
88,CCSFS SLC 40,-80.577366,28.561857,1
89,CCSFS SLC 40,-80.577366,28.561857,1


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`

In [18]:
marker_cluster = MarkerCluster()
circle = folium.Circle(coords, radius=1000, color="#ff0000", fill=True).add_child(folium.Popup(row['LaunchSite']))

*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [19]:
def color(item):
    if item == 1:
        return ("#00ff55")
    elif item == 0:
        return ("#ff0000")

df_spacex['Color'] = df['Class'].apply(color)

df_spacex

C:\Users\brooksat\AppData\Local\Temp\ipykernel_37084\3036768171.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_spacex['Color'] = df['Class'].apply(color)


,LaunchSite,Longitude,Latitude,Class,Color
0,CCSFS SLC 40,-80.577366,28.561857,0,#ff0000
1,CCSFS SLC 40,-80.577366,28.561857,0,#ff0000
2,CCSFS SLC 40,-80.577366,28.561857,0,#ff0000
3,VAFB SLC 4E,-120.610829,34.632093,0,#ff0000
4,CCSFS SLC 40,-80.577366,28.561857,0,#ff0000
...,...,...,...,...,...
85,KSC LC 39A,-80.603956,28.608058,1,#00ff55
86,KSC LC 39A,-80.603956,28.608058,1,#00ff55
87,KSC LC 39A,-80.603956,28.608058,1,#00ff55
88,CCSFS SLC 40,-80.577366,28.561857,1,#00ff55


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`

In [20]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in df_spacex.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    coords = [record['Latitude'], record['Longitude']]
    # marker = folium.Marker(...)
    marker = folium.Marker(
        coords, 
        icon = folium.Icon(color='white', icon_color=record['Color'])
    )     

    marker_cluster.add_child(marker)

site_map

# TASK 3: Calculate the distances between a launch site to its proximities

In [21]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [22]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.

In [24]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
launch_site_lat = 28.56367
launch_site_lon = -80.57163
coastline_lat = 28.6121
coastline_lon = -80.5972
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
distance_coastline

5.937615445695327

In [26]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
coordinate = [coastline_lat, coastline_lon]
distance_marker = folium.Marker(
    coordinate,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
        )
    )

site_map.add_child(distance_marker)
site_map

In [33]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
coordinates = [[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]]
lines=folium.PolyLine(locations=coordinates, weight=1)

site_map.add_child(lines)